---
format: 
  html:
    toc: true
execute:
  echo: true
  code-fold: true
---

# Data Collection & Preprocessing

We begin by importing socio-demographic data from the American Community Survey and creating a new dataframe.

In [1]:
#| echo: true
#| code-fold: true
import pandas as pd
import geopandas as gpd
from census import Census
from us import states
import matplotlib.pyplot as plt

API_KEY = '0da0c882151e10740c1a0a844cf845096bedb565'
c = Census(API_KEY)

acs_data = c.acs5.state_county(
    fields=[
        'NAME',         # County name
        'B01003_001E',  # Total population
        'B15003_001E',  # Population 25+
        'B19013_001E',  # Median household income
        'B19301_001E',  # Per capita income
        'B01002_001E',  # Median age
        'B02001_002E',  # White population
        'B02001_003E',  # African American population
        'B03001_003E',  # Hispanic population
        'B15003_017E',  # High school graduates
        'B15003_022E',  # Bachelor’s degree holders
        'B25077_001E',  # Median housing value
        'B17001_002E',  # Population below poverty line
    ],
    state_fips=states.PA.fips,
    county_fips="*"
)

acs_df = pd.DataFrame(acs_data)
acs_df.columns = [
    'County',  # Name of the county
    'Total Population',
    'Population Over 25',
    'Median Household Income', 
    'Per Capita Income', 
    'Median Age', 
    'White Population', 
    'African American Population', 
    'Hispanic Population', 
    'High School Graduates', 
    'Bachelors Degree Holders', 
    'Median Housing Value', 
    'Population Below Poverty Line',
    'State',  
    'County_Name'
]

acs_df['County'] = acs_df['County'].str.replace(" County, Pennsylvania", "", case=False)
acs_df['County'] = acs_df['County'].str.upper()
acs_df.drop(columns=['State'], inplace=True)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Next, we load our spatial data: Pennsylvania counties. We will be analyzing 67 counties in the scope of this project. We create additional variables in our dataframe for race and educuation data.

In [2]:
#| echo: true
#| code-fold: true
geojson_path = "/Users/ryanswett/Downloads/Python/Final_Project/PaCounty2024_11.geojson"
county_data = gpd.read_file(geojson_path)
counties = county_data.merge(
    acs_df,
    left_on='FIPS_COUNT',
    right_on='County_Name',
    how='left'
)

#| echo: true
#| code-fold: true
counties['Percent_White'] = counties['White Population'] / counties['Total Population']*100 # Percent white
counties['Percent_Black'] = counties['African American Population'] / counties['Total Population']*100 # Percent black
counties['Percent_Hispanic'] = counties['Hispanic Population'] / counties['Total Population']*100 # Percent hispanic
counties['Percent_HS_degrees'] = counties['High School Graduates'] / counties['Total Population']*100 # Percent high school grads
counties['Percent_Bachelors'] = counties['Bachelors Degree Holders'] / counties['Population Over 25']*100 # Percent bachelors
counties['Percent_Poverty'] = counties['Population Below Poverty Line'] / counties['Population Over 25']*100 # Percent below poverty line

Next, the park data is loaded into the project and cleaned by removing columns and calculating a new column for percent local park of each county.

In [4]:
#| echo: true
#| code-fold: true
parks_path = "/Users/ryanswett/Downloads/Python/Final_Project/DCNR_LocalPark202406/DCNR_LocalPark202406.shp"
parks_data = gpd.read_file(parks_path)

parks_data = parks_data.drop(columns=['STATUS', 'PARK_FEE', 'ALT_NAME', 'PREMISE_AD', 'PREMISE_CI', 'PREMISE_ZI', 'YEAR_OPEN',
                                     'PREMISE_CR', 'URL', 'COMMENTS', 'ATV', 'Basketball', 'Bicycling', 'Camping', 'Canoeing_K',
                                     'CrossCount', 'Disc_Golf', 'Dog_Park', 'Equestrian', 'Fishing', 'Fitness_Eq', 'Golf',
                                     'Hiking', 'Horseback_', 'Hunting', 'Ice_Fishin', 'Ice_Skatin', 'Motor_Boat', 'LWCF_Restr',
                                     'Mountain_B', 'Natural_Wi', 'Organized_', 'Parking', 'Pavilion', 'Pets_Allow', 'Playground',
                                     'Restrooms', 'Rock_Climb', 'Scenic_Vie', 'Sledding', 'Sports_Fie', 'Swimming', 'Tennis_Cou',
                                     'Theatre_Am', 'Trails', 'Visitor_Ce', 'White_Wate', 'Wildlife_W', 'Amenity_Co', 'Feedback_l',
                                     'Skate_Park'])

updated_parks = parks_data.groupby('PREMISE_CO')['Acres'].sum().reset_index()
updated_parks['park_sq_mi'] = updated_parks['Acres'] / 640
updated_parks['PREMISE_CO'] = updated_parks['PREMISE_CO'].str.upper()

parks = counties.merge(
    updated_parks,
    left_on='County',  
    right_on='PREMISE_CO',  
    how='left')

parks['Percent Local Park'] = parks['park_sq_mi'] / parks['AREA_SQ_MI'] * 100

Finally, we add the unemployment data to the project and print the final dataframe with all of the data, called *pa_counties*. The final dataframe contains 67 rows, one for each Pennsylvania county. It will be used throughout the ESDA and K-Means Clustering sections to create spatial visualizations and draw informed conclusions.

In [5]:
#| echo: true
#| code-fold: true
unemp_data = pd.read_csv("/Users/ryanswett/Downloads/Python/Final_Project/unemp.csv")
unemp_data['County'] = unemp_data['County'].str.replace('County', '', case=False).str.strip()
unemp_data['County'] = unemp_data['County'].str.upper()

pa_counties = unemp_data.merge(
    parks,
    left_on='County', 
    right_on='County', 
    how='left')

pa_counties.rename(columns={"Value (Percent)": "Unemp Rate"}, inplace=True)

pd.set_option('display.max_columns', None)
pa_counties.head()

County     FIPS  Unemp Rate SOUND  MSLINK MAINT_CTY_  AREA  \
0  PHILADELPHIA  42101.0         8.6   NaN    67.0          5   0.0   
1          PIKE  42103.0         8.5   NaN    51.0          4   0.0   
2       FAYETTE  42051.0         7.2   NaN    26.0          1   0.0   
3       CAMERON  42023.0         7.1   NaN    12.0          4   0.0   
4        CARBON  42025.0         6.9   NaN    13.0          2   0.0   

   COUNTY_PER  COUNTY_N_1 NOTE_FILE IMAGE_NAME  LEN  NUMERIC_LA  AREA_SQ_MI  \
0         0.0        67.0       NaN   poll.bmp  0.0         2.0  143.491659   
1         0.0        51.0       NaN   poll.bmp  0.0         2.0  567.997536   
2         0.0        26.0       NaN   poll.bmp  0.0         1.0  798.810630   
3         0.0        12.0       NaN   poll.bmp  0.0         2.0  399.572096   
4         0.0        13.0       NaN   poll.bmp  0.0         2.0  388.633376   

     GPID PA_CTY_COD     Shape_Leng    COUNTY_NAM DISTRICT_O DISTRICT_N  \
0  1785.0         51  157483.250595  PHILADELPHIA        6-5         06   
1  1759.0         52  293195.463955          PIKE        4-4         04   
2  1793.0         26  319645.320154       FAYETTE       12-1         12   
3  1768.0         12  193699.130807       CAMERON        2-4         02   
4  1769.0         13  207463.124915        CARBON        5-2         05   

  SPREAD_SHE COUNTY_NUM FIPS_COUNT  COUNTY_ARE  \
0        NaN         67        101         0.0   
1        NaN         51        103         0.0   
2        NaN         26        051         0.0   
3        NaN         12        023         0.0   
4        NaN         13        025         0.0   

                                            geometry  Total Population  \
0  POLYGON ((-75.23643 39.93505, -75.23639 39.935...         1593208.0   
1  POLYGON ((-75.17105 41.47278, -75.17085 41.472...           58996.0   
2  POLYGON ((-79.92559 39.91495, -79.92527 39.915...          128417.0   
3  POLYGON ((-78.09337 41.21718, -78.11147 41.218...            4536.0   
4  POLYGON ((-75.60908 40.78745, -75.67682 40.774...           65018.0   

   Population Over 25  Median Household Income  Per Capita Income  Median Age  \
0           1090012.0                  57537.0            35553.0        34.8   
1             44410.0                  76416.0            41139.0        48.9   
2             94625.0                  55579.0            31498.0        45.2   
3              3476.0                  46186.0            27426.0        51.8   
4             48059.0                  64538.0            34375.0        46.3   

   White Population  African American Population  Hispanic Population  \
0          590860.0                     638907.0             249723.0   
1           48549.0                       3568.0               7048.0   
2          116188.0                       5437.0               1780.0   
3            4311.0                          2.0                 91.0   
4           59656.0                       1582.0               3889.0   

   High School Graduates  Bachelors Degree Holders  Median Housing Value  \
0               287402.0                  204967.0              215500.0   
1                12916.0                    6914.0              225100.0   
2                39796.0                   11625.0              121800.0   
3                 1604.0                     302.0               85900.0   
4                19099.0                    5999.0              179600.0   

   Population Below Poverty Line County_Name  Percent_White  Percent_Black  \
0                       351811.0         101      37.086181      40.101920   
1                         5854.0         103      82.292020       6.047868   
2                        20090.0         051      90.477118       4.233863   
3                          674.0         023      95.039683       0.044092   
4                         7551.0         025      91.753053       2.433172   

   Percent_Hispanic  Percent_HS_degrees  Percent_Bachelors